# EfficientNet-B3 Transfer Learning for ISIC 2019 Skin Lesion Classification

## Medical Image Classification with Progressive Training

This notebook implements EfficientNet-B3 transfer learning with progressive training for automated skin lesion diagnosis using the ISIC 2019 dataset.

### Key Features:
- **Progressive Training**: Two-stage approach (frozen backbone → full fine-tuning)
- **EfficientNet-B3**: State-of-the-art efficient architecture optimized for medical images
- **Enhanced Augmentation**: Medical-specific data augmentation pipeline
- **Cosine Annealing**: Warm restart scheduler for improved convergence
- **Class Balancing**: Handles imbalanced medical data

### Architecture Overview:
```
Input (300x300) → EfficientNet-B3 Features → Global Avg Pool → Custom Classifier → 9 Classes
                      ↓ Progressive Training ↓
                 Stage 1: Frozen → Stage 2: Fine-tuned
```

In [ ]:
%cd /content/notebooks/

In [ ]:
import os
import sys

print(f"Current working directory: {os.getcwd()}")

scripts_dir = os.path.abspath('../scripts')
print(f"Attempting to add {scripts_dir} to sys.path")
sys.path.append(scripts_dir)

try:
    print(f"Listing contents of {scripts_dir}:")
    if os.path.exists(scripts_dir):
        print(os.listdir(scripts_dir))
    else:
        print(f"Error: Directory not found at {scripts_dir}")

    # Attempt to import modules
    from data_loader import AzureBlobLoader
    from image_preprocessor import ImagePreprocessor
    print("\nSuccessfully imported data_loader and image_preprocessor!")

except ModuleNotFoundError as e:
    print(f"\nImport Error: {e}")
    print("Please ensure the 'scripts' directory exists one level up from the notebook,")
    print("and that 'data_loader.py' and 'image_preprocessor.py' are present in that directory.")
except ImportError as e:
    print(f"\nImport Error: {e}")
    print("There might be an issue within the data_loader.py or image_preprocessor.py files.")
    print("Please check the content of these script files for errors.")
except Exception as e:
    print(f"\nAn unexpected error occurred during import: {e}")

## Setup and Configuration

In [ ]:
# Essential imports
import os
import sys
import json
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Deep learning imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms, models
from torchvision.utils import make_grid

# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight

# Add scripts directory to path
sys.path.append('../scripts')
from data_loader import AzureBlobLoader
from image_preprocessor import ImagePreprocessor

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

print("Setup complete!")

In [ ]:
# # Configuration constants
# CONFIG = {
#     'azure_account': 'w281saysxxfypm',
#     'azure_container': 'isic-2019-data',
#     'data_dir': '../data',
#     'images_dir': '../data/isic_images',
#     'metadata_file': 'ISIC_2019_Training_Metadata.csv',
#     'ground_truth_file': 'ISIC_2019_Training_GroundTruth.csv',
#     'num_images': 5000,  # For initial testing
#     'batch_size': 32,
#     'num_epochs': 50,
#     'learning_rate': 0.001,
#     'weight_decay': 1e-4,
#     'num_workers': 4,
#     'patience': 10,  # Early stopping
#     'validation_split': 0.2,
#     'test_split': 0.1,
#     'image_size': 224,  # VGG16 input size
#     'num_classes': 9,
#     'class_names': ['MEL', 'NV', 'BCC', 'AK', 'BKL', 'DF', 'VASC', 'SCC', 'UNK']
# }

# print("Configuration loaded:")
# for key, value in CONFIG.items():
#     print(f"  {key}: {value}")

In [ ]:
# Updated configuration for EfficientNet-B3 with progressive training
CONFIG = {
    'azure_account': 'w281saysxxfypm',
    'azure_container': 'isic-2019-data',
    'data_dir': '../data',
    'images_dir': '../data/isic_images',
    'metadata_file': 'ISIC_2019_Training_Metadata.csv',
    'ground_truth_file': 'ISIC_2019_Training_GroundTruth.csv',
    'num_images': 8000,  # Increased for better EfficientNet performance
    'batch_size': 16,    # Reduced for EfficientNet-B3 memory requirements
    'num_epochs': 60,    # Increased for progressive training
    'learning_rate': 5e-4,  # Lower learning rate for EfficientNet
    'weight_decay': 1e-5,   # Reduced weight decay
    'num_workers': 4,
    'patience': 15,      # Increased patience for progressive training
    'validation_split': 0.2,
    'test_split': 0.1,
    'image_size': 300,   # EfficientNet-B3 native size (can also use 224)
    'num_classes': 9,
    'class_names': ['MEL', 'NV', 'BCC', 'AK', 'BKL', 'DF', 'VASC', 'SCC', 'UNK'],
    
    # New progressive training parameters
    'stage1_epochs': 20,
    'backbone_lr_factor': 0.1,  # Backbone learns 10x slower than classifier
    'cosine_restart_period': 10,
    'min_lr': 1e-7,
    
    # Enhanced augmentation parameters
    'tta_enabled': True,  # Test Time Augmentation
    'mixup_alpha': 0.2,   # For future Mixup implementation
    'cutmix_prob': 0.3,   # For future CutMix implementation
}

print("Enhanced configuration loaded:")
for key, value in CONFIG.items():
    print(f"  {key}: {value}")

## Data Loading and Preparation

In [ ]:
# Initialize data loader and preprocessor
data_loader = AzureBlobLoader(CONFIG['azure_account'])
preprocessor = ImagePreprocessor()

# Create directories if they don't exist
os.makedirs(CONFIG['data_dir'], exist_ok=True)
os.makedirs(CONFIG['images_dir'], exist_ok=True)

# Load metadata directly from Azure (no need for local download first)
print("Loading metadata from Azure...")
try:
    # Load metadata using the correct metadata_type parameters
    metadata_df = data_loader.load_metadata('isic', 'training_metadata')
    ground_truth_df = data_loader.load_metadata('isic', 'ground_truth')

    # Save locally for future use
    metadata_path = os.path.join(CONFIG['data_dir'], CONFIG['metadata_file'])
    ground_truth_path = os.path.join(CONFIG['data_dir'], CONFIG['ground_truth_file'])

    metadata_df.to_csv(metadata_path, index=False)
    ground_truth_df.to_csv(ground_truth_path, index=False)
    print("Metadata loaded and saved locally!")

except Exception as e:
    print(f"Error loading metadata from Azure: {e}")
    print("Trying to load from local files...")

    # Fallback to local files
    metadata_path = os.path.join(CONFIG['data_dir'], CONFIG['metadata_file'])
    ground_truth_path = os.path.join(CONFIG['data_dir'], CONFIG['ground_truth_file'])

    if os.path.exists(metadata_path) and os.path.exists(ground_truth_path):
        metadata_df = pd.read_csv(metadata_path)
        ground_truth_df = pd.read_csv(ground_truth_path)
        print("Loaded metadata from local files!")
    else:
        print("No local metadata files found. Please check your data setup.")
        raise FileNotFoundError("Could not load metadata from Azure or local files")

# Merge metadata with ground truth
df = pd.merge(metadata_df, ground_truth_df, on='image')
print(f"Loaded {len(df)} images with metadata")
print("\nDataset columns:", df.columns.tolist())
print("\nClass distribution:")
diagnosis_cols = [col for col in df.columns if col not in ['image', 'age_approx', 'anatom_site_general', 'sex']]

# Fix: Ensure diagnosis columns are numeric before summing
for col in diagnosis_cols:
    # Convert to numeric, replacing non-numeric values with 0
    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)
    count = df[col].sum()
    if count > 0:
        print(f"  {col}: {count}")

In [ ]:
# Sample data for training (to speed up development)
print(f"Sampling {CONFIG['num_images']} images for training...")

# Create target column for classification
df['target'] = df[diagnosis_cols].idxmax(axis=1)
df['target_idx'] = LabelEncoder().fit_transform(df['target'])

# Stratified sampling to maintain class distribution
from sklearn.model_selection import train_test_split
sampled_df, _ = train_test_split(
    df,
    train_size=CONFIG['num_images'],
    stratify=df['target'],
    random_state=42
)

print(f"Sampled dataset class distribution:")
for i, class_name in enumerate(CONFIG['class_names']):
    count = (sampled_df['target_idx'] == i).sum()
    print(f"  {class_name}: {count}")

# Check which images need to be downloaded
existing_images = set()
if os.path.exists(CONFIG['images_dir']):
    existing_images = set(os.listdir(CONFIG['images_dir']))
    existing_images = {img.replace('.jpg', '') for img in existing_images if img.endswith('.jpg')}

images_to_download = []
for image_id in sampled_df['image']:
    if image_id not in existing_images:
        images_to_download.append(image_id)

print(f"Found {len(existing_images)} existing images")
print(f"Need to download {len(images_to_download)} new images")

# Download missing images using the correct method
if images_to_download:
    print("Downloading images... This may take a while.")
    try:
        successful, failed = data_loader.download_batch(
            images_to_download,
            dataset='isic',
            local_dir=CONFIG['images_dir'],
            max_workers=5
        )
        print(f"Downloaded {len(successful)} images, {len(failed)} errors")

        if failed:
            print(f"Failed to download: {failed[:5]}...")  # Show first 5 failures
    except Exception as e:
        print(f"Error during batch download: {e}")
        print("You may need to check your network connection or Azure access.")
else:
    print("All required images already downloaded!")

## Custom Dataset and Data Transforms

In [ ]:
from torch.utils.data import Dataset, DataLoader # Import Dataset and DataLoader
import os
import numpy as np
from PIL import Image
import torch
import torchvision.transforms as transforms # Import transforms here for fallback

class SkinLesionDataset(Dataset):
    """Custom dataset for skin lesion images with advanced preprocessing"""

    def __init__(self, dataframe, images_dir, transform=None, preprocessing=True):
        # Create a clean copy of the dataframe and reset index
        self.df = dataframe.copy().reset_index(drop=True)
        self.images_dir = images_dir
        self.transform = transform
        self.preprocessing = preprocessing

        # Ensure ImagePreprocessor is imported if not already available
        try:
            from image_preprocessor import ImagePreprocessor
            self.preprocessor = ImagePreprocessor()
        except ImportError:
            print("Warning: ImagePreprocessor not found. Preprocessing will be disabled.")
            self.preprocessor = None
            self.preprocessing = False

        # Filter only images that exist and ensure target_idx column exists
        self.df = self._filter_existing_images()
        self._validate_columns() # Ensure target_idx exists
        print(f"Dataset initialized with {len(self.df)} valid images")

    def _validate_columns(self):
        """Ensure required columns exist in the dataframe"""
        required_columns = ['image', 'target_idx']
        for col in required_columns:
            if col not in self.df.columns:
                if col == 'target_idx' and 'target' in self.df.columns:
                    # Create target_idx from target if it doesn't exist
                    from sklearn.preprocessing import LabelEncoder
                    le = LabelEncoder()
                    self.df['target_idx'] = le.fit_transform(self.df['target'])
                    print(f"Created {col} column from 'target' column")
                else:
                    raise ValueError(f"Required column '{col}' not found in dataframe. Available columns: {list(self.df.columns)}")


    def _filter_existing_images(self):
        """Filter dataframe to only include images that exist on disk"""
        print(f"Filtering {len(self.df)} images for existing files...")

        # Create a boolean mask for existing images
        existing_mask = []
        for idx, row in self.df.iterrows():
            image_path = os.path.join(self.images_dir, f"{row['image']}.jpg")
            exists = os.path.exists(image_path)
            existing_mask.append(exists)

            # Debug: print first few missing files
            if not exists and len([x for x in existing_mask if not x]) <= 3:
                print(f"Missing image: {image_path}")


        # Filter the dataframe using the mask - this preserves all columns
        filtered_df = self.df[existing_mask].copy().reset_index(drop=True)

        missing_count = len(self.df) - len(filtered_df)
        if missing_count > 0:
            print(f"Warning: {missing_count} images not found on disk and will be skipped")

        return filtered_df

    # Add the __len__ method
    def __len__(self):
        """Returns the number of samples in the dataset"""
        return len(self.df)

    def __getitem__(self, idx):
        try:
            # Get image info
            row = self.df.iloc[idx]
            image_id = row['image']

            # Ensure target_idx exists and is valid
            if 'target_idx' not in row:
                raise KeyError(f"target_idx column not found. Available columns: {list(row.index)}")

            label = int(row['target_idx'])

            # Load image
            image_path = os.path.join(self.images_dir, f"{image_id}.jpg")

            if not os.path.exists(image_path):
                raise FileNotFoundError(f"Image not found: {image_path}")

            # Load image using PIL
            image = Image.open(image_path).convert('RGB')

            # Apply custom preprocessing if enabled and preprocessor is available
            if self.preprocessing and self.preprocessor:
                try:
                    # Convert PIL to numpy for preprocessing
                    image_np = np.array(image)

                    # The detect_circular_vignette method returns a tuple (image_array, int, int)
                    # FIXED: Always unpack the tuple properly
                    processed_result = self.preprocessor.detect_circular_vignette(image_np)

                    # The method always returns a tuple (image_array, width, height)
                    if isinstance(processed_result, tuple) and len(processed_result) >= 1:
                        processed_image = processed_result[0]  # Get the image array
                    else:
                        # Fallback to original image
                        processed_image = image_np
                        print(f"Warning: Unexpected preprocessor output for {image_id}, using original")

                    # Ensure the processed image is valid
                    if processed_image is None or processed_image.size == 0:
                         processed_image = image_np
                         print(f"Warning: Empty processed image for {image_id}, using original")


                    # Convert back to PIL
                    image = Image.fromarray(processed_image.astype(np.uint8))

                except Exception as e:
                    print(f"Warning: Preprocessing failed for {image_id}: {e}")
                    # Continue with original image if preprocessing fails

            # Apply transforms
            if self.transform:
                try:
                    image = self.transform(image)
                except Exception as e:
                    print(f"Warning: Transform failed for {image_id}: {e}")
                    # Create a fallback tensor
                    image = torch.zeros(3, 224, 224)
            else:
                # Convert to tensor if no transform provided
                # Use the transforms import from the top of the cell
                image = transforms.ToTensor()(image)

            # Final validation - ensure the image is a proper tensor
            if not isinstance(image, torch.Tensor):
                print(f"Warning: Image for {image_id} is not a tensor after transforms")
                image = torch.zeros(3, 224, 224)

            # Check tensor shape (assuming 3 channel, 224x224)
            if image.shape != (3, 224, 224):
                print(f"Warning: Image {image_id} has wrong shape {image.shape}, creating fallback")
                image = torch.zeros(3, 224, 224)


            return image, label, image_id

        except FileNotFoundError:
             # Handle file not found specifically
             print(f"Error: Image file not found at {image_path} for index {idx}. Returning black image.")
             # Return a black image and a random label as fallback for file not found
             black_image = torch.zeros(3, 224, 224)
             # Use a random label to avoid biasing towards class 0
             fallback_label = np.random.randint(0, len(CONFIG['class_names'])) # Use CONFIG from global scope
             fallback_id = f"missing_image_{idx}"
             return black_image, fallback_label, fallback_id


        except Exception as e:
            print(f"Error processing image at index {idx} with ID {image_id if 'image_id' in locals() else 'N/A'}: {e}")
            # Return a black image and a random label for other errors
            black_image = torch.zeros(3, 224, 224)
            # Use a random label to avoid biasing towards class 0
            fallback_label = np.random.randint(0, len(CONFIG['class_names'])) # Use CONFIG from global scope
            fallback_id = f"error_processing_{idx}"


            return black_image, fallback_label, fallback_id

print("SkinLesionDataset class defined with error handling and fallback!")

In [ ]:
# # Define data transforms optimized for medical images
# train_transforms = transforms.Compose([
#     transforms.Resize((CONFIG['image_size'], CONFIG['image_size'])),
#     transforms.RandomHorizontalFlip(p=0.5),
#     transforms.RandomVerticalFlip(p=0.5),
#     transforms.RandomRotation(degrees=20),
#     transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
#     transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.9, 1.1)),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet normalization
# ])

# val_transforms = transforms.Compose([
#     transforms.Resize((CONFIG['image_size'], CONFIG['image_size'])),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
# ])

# print("Data transforms defined!")
# print("Training transforms include:")
# print("  - Resize to 224x224")
# print("  - Random horizontal/vertical flips")
# print("  - Random rotation (±20°)")
# print("  - Color jittering")
# print("  - Random affine transformations")
# print("  - ImageNet normalization")
# print("\nValidation transforms include:")
# print("  - Resize to 224x224")
# print("  - ImageNet normalization only")

### Enhanced data augmentation optimized for medical images


In [ ]:
from torchvision.transforms import InterpolationMode

# Advanced training transforms with medical-specific augmentations
train_transforms = transforms.Compose([
    # Resize with some padding for better random crops
    transforms.Resize((CONFIG['image_size'] + 32, CONFIG['image_size'] + 32), 
                     interpolation=InterpolationMode.BILINEAR),
    
    # Random resized crop with medical-appropriate scale
    transforms.RandomResizedCrop(CONFIG['image_size'], 
                                scale=(0.8, 1.0), 
                                ratio=(0.9, 1.1),
                                interpolation=InterpolationMode.BILINEAR),
    
    # Medical image specific flips
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    
    # Enhanced rotation for skin lesions
    transforms.RandomRotation(degrees=45, interpolation=InterpolationMode.BILINEAR),
    
    # Advanced color augmentation for medical images
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.15),
    
    # Affine transformations
    transforms.RandomAffine(degrees=0, 
                           translate=(0.1, 0.1), 
                           scale=(0.85, 1.15),
                           shear=10,
                           interpolation=InterpolationMode.BILINEAR),
    
    # Advanced augmentations
    transforms.RandomApply([
        transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 2.0))
    ], p=0.3),
    
    transforms.RandomApply([
        transforms.RandomAdjustSharpness(sharpness_factor=2)
    ], p=0.3),
    
    # Convert to tensor
    transforms.ToTensor(),
    
    # ImageNet normalization
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    
    # Random erasing (cutout) - very effective for medical images
    transforms.RandomErasing(p=0.25, scale=(0.02, 0.15), ratio=(0.3, 3.3), value=0)
])

# Test Time Augmentation transforms (for inference)
tta_transforms = [
    # Original
    transforms.Compose([
        transforms.Resize((CONFIG['image_size'], CONFIG['image_size'])),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    
    # Horizontal flip
    transforms.Compose([
        transforms.Resize((CONFIG['image_size'], CONFIG['image_size'])),
        transforms.RandomHorizontalFlip(p=1.0),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    
    # Vertical flip
    transforms.Compose([
        transforms.Resize((CONFIG['image_size'], CONFIG['image_size'])),
        transforms.RandomVerticalFlip(p=1.0),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    
    # Slight rotation
    transforms.Compose([
        transforms.Resize((CONFIG['image_size'], CONFIG['image_size'])),
        transforms.RandomRotation(degrees=15),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
]

# Validation transforms remain simple
val_transforms = transforms.Compose([
    transforms.Resize((CONFIG['image_size'], CONFIG['image_size'])),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

print("Enhanced data transforms defined!")
print("Training transforms include:")
print("  - Random resized crop with medical-appropriate scale")
print("  - Enhanced rotation (±45°) and affine transformations")
print("  - Advanced color jittering optimized for skin lesions")
print("  - Gaussian blur and sharpness adjustments")
print("  - Random erasing (cutout) for regularization")
print("  - Medical-specific flip augmentations")
print("\nTest Time Augmentation (TTA) transforms available for inference")

In [ ]:
# Create train/validation/test splits
print("Creating train/validation/test splits...")

# First split: separate test set
train_val_df, test_df = train_test_split(
    sampled_df,
    test_size=CONFIG['test_split'],
    stratify=sampled_df['target_idx'],
    random_state=42
)

# Second split: separate train and validation
train_df, val_df = train_test_split(
    train_val_df,
    test_size=CONFIG['validation_split']/(1-CONFIG['test_split']),
    stratify=train_val_df['target_idx'],
    random_state=42
)

print(f"Dataset splits:")
print(f"  Training: {len(train_df)} images")
print(f"  Validation: {len(val_df)} images")
print(f"  Test: {len(test_df)} images")

# Create datasets
train_dataset = SkinLesionDataset(train_df, CONFIG['images_dir'], train_transforms, preprocessing=True)
val_dataset = SkinLesionDataset(val_df, CONFIG['images_dir'], val_transforms, preprocessing=True)
test_dataset = SkinLesionDataset(test_df, CONFIG['images_dir'], val_transforms, preprocessing=True)

# Create data loaders
train_loader = DataLoader(
    train_dataset,
    batch_size=CONFIG['batch_size'],
    shuffle=True,
    num_workers=CONFIG['num_workers'],
    pin_memory=True if device.type == 'cuda' else False
)

val_loader = DataLoader(
    val_dataset,
    batch_size=CONFIG['batch_size'],
    shuffle=False,
    num_workers=CONFIG['num_workers'],
    pin_memory=True if device.type == 'cuda' else False
)

test_loader = DataLoader(
    test_dataset,
    batch_size=CONFIG['batch_size'],
    shuffle=False,
    num_workers=CONFIG['num_workers'],
    pin_memory=True if device.type == 'cuda' else False
)

print(f"Data loaders created!")
print(f"Batch size: {CONFIG['batch_size']}")
print(f"Number of workers: {CONFIG['num_workers']}")
print(f"Pin memory: {device.type == 'cuda'}")

## Data Visualization and Analysis

In [ ]:
# Visualize class distribution
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

datasets = [('Training', train_df), ('Validation', val_df), ('Test', test_df)]
colors = plt.cm.Set3(np.linspace(0, 1, len(CONFIG['class_names'])))

for idx, (name, df) in enumerate(datasets):
    # Ensure all possible class indices are considered, even if some have 0 counts
    class_counts = [sum(df['target_idx'] == i) for i in range(len(CONFIG['class_names']))]

    axes[idx].bar(CONFIG['class_names'], class_counts, color=colors)
    axes[idx].set_title(f'{name} Set Class Distribution')
    axes[idx].set_xlabel('Diagnosis')
    axes[idx].set_ylabel('Count')
    axes[idx].tick_params(axis='x', rotation=45)

    # Add count labels on bars
    for i, count in enumerate(class_counts):
        # Only add label if count is greater than 0
        if count > 0:
            axes[idx].text(i, count + 0.5, str(count), ha='center', va='bottom')

plt.tight_layout()
plt.show()

# Calculate class weights for handling imbalanced data
unique_classes_in_train = np.unique(train_df['target_idx'])
print(f"Classes present in training: {unique_classes_in_train}")

# Only calculate weights for classes that exist in training
if len(unique_classes_in_train) > 1:
    class_weights = compute_class_weight('balanced', classes=unique_classes_in_train, y=train_df['target_idx'])

    # Create a full weights tensor for all 9 classes
    full_class_weights = np.ones(len(CONFIG['class_names']))  # Start with 1.0 instead of 0
    for class_idx, weight in zip(unique_classes_in_train, class_weights):
        full_class_weights[class_idx] = weight

    # For missing classes, use a small weight instead of 0
    for i in range(len(CONFIG['class_names'])):
        if i not in unique_classes_in_train:
            full_class_weights[i] = 0.1  # Small weight for missing classes
else:
    # Fallback: equal weights
    full_class_weights = np.ones(len(CONFIG['class_names']))

class_weights_tensor = torch.FloatTensor(full_class_weights).to(device)

print("Class weights for balanced training:")
train_class_counts = np.bincount(train_df['target_idx'], minlength=len(CONFIG['class_names']))
for i, (name, weight) in enumerate(zip(CONFIG['class_names'], full_class_weights)):
    print(f"  {name}: {weight:.3f} (count: {train_class_counts[i]})")

In [ ]:
# Visualize sample images from each class
def denormalize(tensor):
    """Denormalize tensor for visualization"""
    mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
    std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)
    return tensor * std + mean

def show_sample_images():
    """Display sample images from each diagnostic class"""
    fig, axes = plt.subplots(3, 3, figsize=(15, 15))
    axes = axes.flatten()

    # Get one sample from each class
    for class_idx in range(min(9, len(CONFIG['class_names']))):
        class_samples = train_df[train_df['target_idx'] == class_idx]
        if len(class_samples) > 0:
            # Get random sample
            sample_idx = np.random.randint(0, len(class_samples))
            sample_row = class_samples.iloc[sample_idx]

            # Load and process image
            image_path = os.path.join(CONFIG['images_dir'], f"{sample_row['image']}.jpg")
            if os.path.exists(image_path):
                from PIL import Image
                image = Image.open(image_path).convert('RGB')

                # Apply preprocessing and transforms
                preprocessor = ImagePreprocessor()
                image_np = np.array(image)

                # The detect_circular_vignette method returns a tuple (image_array, int, int)
                # Unpack the tuple and use only the image array
                processed_image_tuple = preprocessor.detect_circular_vignette(image_np)
                processed_image = processed_image_tuple[0] # Get the image array

                image_pil = Image.fromarray(processed_image.astype(np.uint8))

                # Apply validation transforms (no augmentation)
                image_tensor = val_transforms(image_pil)

                # Denormalize for display
                display_image = denormalize(image_tensor)
                display_image = torch.clamp(display_image, 0, 1)

                # Convert to numpy and transpose for matplotlib
                display_image = display_image.permute(1, 2, 0).numpy()

                axes[class_idx].imshow(display_image)
                axes[class_idx].set_title(f'{CONFIG["class_names"][class_idx]}\n({sample_row["image"]})')
                axes[class_idx].axis('off')
            else:
                axes[class_idx].text(0.5, 0.5, f'{CONFIG["class_names"][class_idx]}\nImage not found',
                                   ha='center', va='center', transform=axes[class_idx].transAxes)
                axes[class_idx].axis('off')
        else:
            axes[class_idx].text(0.5, 0.5, f'{CONFIG["class_names"][class_idx]}\nNo samples',
                               ha='center', va='center', transform=axes[class_idx].transAxes)
            axes[class_idx].axis('off')

    plt.suptitle('Sample Images from Each Diagnostic Class (After Preprocessing)', fontsize=16)
    plt.tight_layout()
    plt.show()

show_sample_images()

## VGG16 Transfer Learning Model

In [ ]:
# class VGG16SkinLesionClassifier(nn.Module):
#     """VGG16-based transfer learning model for skin lesion classification"""

#     def __init__(self, num_classes=9, pretrained=True, freeze_features=True, dropout_rate=0.5):
#         super(VGG16SkinLesionClassifier, self).__init__()

#         # Load pretrained VGG16
#         self.vgg16 = models.vgg16(pretrained=pretrained)

#         # Freeze feature extraction layers if specified
#         if freeze_features:
#             for param in self.vgg16.features.parameters():
#                 param.requires_grad = False
#             print("VGG16 feature layers frozen")
#         else:
#             print("VGG16 feature layers unfrozen")

#         # Get number of features from the last layer before classifier
#         num_features = self.vgg16.classifier[6].in_features

#         # Replace the classifier with our custom one
#         self.vgg16.classifier = nn.Sequential(
#             nn.Linear(25088, 4096),
#             nn.ReLU(True),
#             nn.Dropout(dropout_rate),
#             nn.Linear(4096, 2048),
#             nn.ReLU(True),
#             nn.Dropout(dropout_rate),
#             nn.Linear(2048, 1024),
#             nn.ReLU(True),
#             nn.Dropout(dropout_rate),
#             nn.Linear(1024, 512),
#             nn.ReLU(True),
#             nn.Dropout(dropout_rate),
#             nn.Linear(512, num_classes)
#         )

#         # Store feature extractor (all layers except the final classifier)
#         self.feature_extractor = nn.Sequential(*list(self.vgg16.children())[:-1])
#         self.avgpool = nn.AdaptiveAvgPool2d((7, 7))

#         self.num_classes = num_classes
#         self.dropout_rate = dropout_rate

#     def forward(self, x):
#         """Forward pass"""
#         return self.vgg16(x)

#     def extract_features(self, x):
#         """Extract features from the second-to-last layer"""
#         # Pass through feature extraction layers
#         x = self.vgg16.features(x)
#         x = self.avgpool(x)
#         x = torch.flatten(x, 1)

#         # Pass through classifier layers except the last one
#         classifier_layers = list(self.vgg16.classifier.children())[:-1]
#         for layer in classifier_layers:
#             x = layer(x)

#         return x

#     def get_model_summary(self):
#         """Print model architecture summary"""
#         total_params = sum(p.numel() for p in self.parameters())
#         trainable_params = sum(p.numel() for p in self.parameters() if p.requires_grad)

#         print("VGG16 Skin Lesion Classifier Architecture:")
#         print(f"  Total parameters: {total_params:,}")
#         print(f"  Trainable parameters: {trainable_params:,}")
#         print(f"  Frozen parameters: {total_params - trainable_params:,}")
#         print(f"  Dropout rate: {self.dropout_rate}")
#         print(f"  Number of classes: {self.num_classes}")

#         return total_params, trainable_params

# # Create model
# model = VGG16SkinLesionClassifier(
#     num_classes=CONFIG['num_classes'],
#     pretrained=True,
#     freeze_features=True,
#     dropout_rate=0.5
# )

# # Move model to device
# model = model.to(device)

# # Get model summary
# total_params, trainable_params = model.get_model_summary()

# print(f"\nModel loaded on: {device}")
# print("VGG16 Transfer Learning Model created!")

## EfficientNet-B3 Implementation


In [ ]:
from efficientnet_pytorch import EfficientNet

class EfficientNetB3SkinLesionClassifier(nn.Module):
    """EfficientNet-B3 based transfer learning model for skin lesion classification"""
    
    def __init__(self, num_classes=9, pretrained=True, dropout_rate=0.3):
        super(EfficientNetB3SkinLesionClassifier, self).__init__()
        
        # Load pretrained EfficientNet-B3
        if pretrained:
            self.efficientnet = EfficientNet.from_pretrained('efficientnet-b3', num_classes=num_classes)
        else:
            self.efficientnet = EfficientNet.from_name('efficientnet-b3', num_classes=num_classes)
        
        # Get the number of features from the classifier
        num_features = self.efficientnet._fc.in_features
        
        # Replace the classifier with custom medical-optimized head
        self.efficientnet._fc = nn.Sequential(
            nn.Dropout(dropout_rate),
            nn.Linear(num_features, 512),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(512),
            nn.Dropout(dropout_rate),
            nn.Linear(512, 256),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(256),
            nn.Dropout(dropout_rate),
            nn.Linear(256, num_classes)
        )
        
        self.num_classes = num_classes
        self.dropout_rate = dropout_rate
        
        # Store the feature extractor layers for feature extraction
        self.feature_layers = nn.Sequential(*list(self.efficientnet._fc.children())[:-1])
    
    def forward(self, x):
        """Forward pass"""
        return self.efficientnet(x)
    
    def extract_features(self, x):
        """Extract features from the second-to-last layer (256-dimensional)"""
        # Pass through the backbone
        x = self.efficientnet.extract_features(x)
        x = self.efficientnet._avg_pooling(x)
        x = x.flatten(start_dim=1)
        x = self.efficientnet._dropout(x)
        
        # Pass through classifier layers except the last one
        feature_layers = list(self.efficientnet._fc.children())[:-1]
        for layer in feature_layers:
            x = layer(x)
        
        return x  # 256-dimensional features
    
    def get_model_summary(self):
        """Print model architecture summary"""
        total_params = sum(p.numel() for p in self.parameters())
        trainable_params = sum(p.numel() for p in self.parameters() if p.requires_grad)
        
        print("EfficientNet-B3 Skin Lesion Classifier Architecture:")
        print(f"  Total parameters: {total_params:,}")
        print(f"  Trainable parameters: {trainable_params:,}")
        print(f"  Frozen parameters: {total_params - trainable_params:,}")
        print(f"  Dropout rate: {self.dropout_rate}")
        print(f"  Number of classes: {self.num_classes}")
        
        return total_params, trainable_params

# Create EfficientNet-B3 model
model = EfficientNetB3SkinLesionClassifier(
    num_classes=CONFIG['num_classes'],
    pretrained=True,
    dropout_rate=0.3
)

# Move model to device
model = model.to(device)

# Get model summary
total_params, trainable_params = model.get_model_summary()

print(f"\nModel loaded on: {device}")
print("EfficientNet-B3 Transfer Learning Model created!")

## Training Setup and Configuration

In [ ]:
# # Loss function with class weights for imbalanced data
# criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)

# # Optimizer - Adam with weight decay
# optimizer = optim.Adam(
#     model.parameters(),
#     lr=CONFIG['learning_rate'],
#     weight_decay=CONFIG['weight_decay']
# )

# # Learning rate scheduler - reduce on plateau
# scheduler = optim.lr_scheduler.ReduceLROnPlateau(
#     optimizer,
#     mode='min',
#     factor=0.5,
#     patience=5,
#     verbose=True,
#     min_lr=1e-7
# )

# # Early stopping setup
# class EarlyStopping:
#     def __init__(self, patience=10, min_delta=0, restore_best_weights=True):
#         self.patience = patience
#         self.min_delta = min_delta
#         self.restore_best_weights = restore_best_weights
#         self.best_loss = None
#         self.counter = 0
#         self.best_weights = None

#     def __call__(self, val_loss, model):
#         if self.best_loss is None:
#             self.best_loss = val_loss
#             self.save_checkpoint(model)
#         elif val_loss < self.best_loss - self.min_delta:
#             self.best_loss = val_loss
#             self.counter = 0
#             self.save_checkpoint(model)
#         else:
#             self.counter += 1

#         if self.counter >= self.patience:
#             if self.restore_best_weights:
#                 model.load_state_dict(self.best_weights)
#             return True
#         return False

#     def save_checkpoint(self, model):
#         self.best_weights = model.state_dict().copy()

# early_stopping = EarlyStopping(patience=CONFIG['patience'])

# print("Training setup complete!")
# print(f"Loss function: CrossEntropyLoss with class weights")
# print(f"Optimizer: Adam (lr={CONFIG['learning_rate']}, wd={CONFIG['weight_decay']})")
# print(f"Scheduler: ReduceLROnPlateau (factor=0.5, patience=5)")
# print(f"Early stopping: Patience={CONFIG['patience']} epochs")
# print(f"Max epochs: {CONFIG['num_epochs']}")

### Progressive Training Setup


In [ ]:
def create_progressive_optimizer_and_scheduler(model, stage='frozen'):
    """Create optimizer and scheduler for different training stages"""
    
    if stage == 'frozen':
        # Stage 1: Only train classifier head
        params_to_train = []
        for name, param in model.named_parameters():
            if '_fc' in name:  # EfficientNet classifier
                param.requires_grad = True
                params_to_train.append(param)
            else:
                param.requires_grad = False
        
        optimizer = optim.AdamW(
            params_to_train,
            lr=CONFIG['learning_rate'],
            weight_decay=CONFIG['weight_decay'],
            betas=(0.9, 0.999)
        )
        
        print(f"Stage 1 (Frozen): Training {sum(p.numel() for p in params_to_train):,} parameters")
        
    else:  # stage == 'fine_tune'
        # Stage 2: Fine-tune entire model with lower learning rate
        for param in model.parameters():
            param.requires_grad = True
        
        # Different learning rates for different parts
        backbone_params = []
        classifier_params = []
        
        for name, param in model.named_parameters():
            if '_fc' in name:
                classifier_params.append(param)
            else:
                backbone_params.append(param)
        
        optimizer = optim.AdamW([
            {'params': backbone_params, 'lr': CONFIG['learning_rate'] * 0.1},
            {'params': classifier_params, 'lr': CONFIG['learning_rate']}
        ], weight_decay=CONFIG['weight_decay'], betas=(0.9, 0.999))
        
        print(f"Stage 2 (Fine-tune): Training all {sum(p.numel() for p in model.parameters()):,} parameters")
    
    # Cosine annealing scheduler with warm restarts
    scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(
        optimizer,
        T_0=10,  # Restart every 10 epochs
        T_mult=2,  # Double restart period each time
        eta_min=1e-7,
        verbose=True
    )
    
    return optimizer, scheduler

# Progressive training function
def train_model_progressive():
    """Progressive training: frozen features → fine-tuning"""
    print("Starting Progressive EfficientNet-B3 Training")
    print("=" * 80)
    
    all_train_losses = []
    all_val_losses = []
    all_train_accs = []
    all_val_accs = []
    
    best_val_acc = 0.0
    total_start_time = datetime.now()
    
    # Stage 1: Frozen backbone training
    print("\nSTAGE 1: Training with frozen backbone (20 epochs)")
    print("-" * 60)
    
    optimizer, scheduler = create_progressive_optimizer_and_scheduler(model, 'frozen')
    criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)
    early_stopping = EarlyStopping(patience=15)
    
    stage1_epochs = 20
    for epoch in range(stage1_epochs):
        epoch_start = datetime.now()
        print(f"\nStage 1 - Epoch {epoch+1}/{stage1_epochs}")
        
        # Training
        train_loss, train_acc, train_preds, train_targets = train_epoch(
            model, train_loader, criterion, optimizer, device, epoch+1
        )
        
        # Validation
        val_loss, val_acc, val_preds, val_targets, val_probs = validate_epoch(
            model, val_loader, criterion, device
        )
        
        # Update scheduler
        scheduler.step(epoch + val_loss / len(val_loader))
        current_lr = optimizer.param_groups[0]['lr']
        
        # Store history
        all_train_losses.append(train_loss)
        all_val_losses.append(val_loss)
        all_train_accs.append(train_acc)
        all_val_accs.append(val_acc)
        
        # Print summary
        epoch_time = datetime.now() - epoch_start
        print(f"  Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
        print(f"  Val Loss:   {val_loss:.4f} | Val Acc:   {val_acc:.2f}%")
        print(f"  LR: {current_lr:.2e} | Time: {epoch_time.total_seconds():.1f}s")
        
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            print(f"New best validation accuracy: {best_val_acc:.2f}%")
        
        if early_stopping(val_loss, model):
            print(f"Early stopping triggered at epoch {epoch+1}")
            break
    
    # Stage 2: Fine-tuning entire model
    print(f"\nSTAGE 2: Fine-tuning entire model ({CONFIG['num_epochs'] - stage1_epochs} epochs)")
    print("-" * 60)
    
    optimizer, scheduler = create_progressive_optimizer_and_scheduler(model, 'fine_tune')
    early_stopping = EarlyStopping(patience=CONFIG['patience'])
    
    remaining_epochs = CONFIG['num_epochs'] - stage1_epochs
    for epoch in range(remaining_epochs):
        epoch_start = datetime.now()
        print(f"\nStage 2 - Epoch {epoch+1}/{remaining_epochs}")
        
        # Training
        train_loss, train_acc, train_preds, train_targets = train_epoch(
            model, train_loader, criterion, optimizer, device, stage1_epochs + epoch + 1
        )
        
        # Validation
        val_loss, val_acc, val_preds, val_targets, val_probs = validate_epoch(
            model, val_loader, criterion, device
        )
        
        # Update scheduler
        scheduler.step(stage1_epochs + epoch + val_loss / len(val_loader))
        
        # Different learning rates for different parameter groups
        backbone_lr = optimizer.param_groups[0]['lr']
        classifier_lr = optimizer.param_groups[1]['lr']
        
        # Store history
        all_train_losses.append(train_loss)
        all_val_losses.append(val_loss)
        all_train_accs.append(train_acc)
        all_val_accs.append(val_acc)
        
        # Print summary
        epoch_time = datetime.now() - epoch_start
        print(f"  Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
        print(f"  Val Loss:   {val_loss:.4f} | Val Acc:   {val_acc:.2f}%")
        print(f"  Backbone LR: {backbone_lr:.2e} | Classifier LR: {classifier_lr:.2e}")
        print(f"  Time: {epoch_time.total_seconds():.1f}s")
        
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            print(f"New best validation accuracy: {best_val_acc:.2f}%")
        
        if early_stopping(val_loss, model):
            print(f"Early stopping triggered at epoch {stage1_epochs + epoch + 1}")
            break
        
        # Detailed metrics every 10 epochs
        if (epoch + 1) % 10 == 0:
            print(f"\nDetailed Validation Metrics (Stage 2, Epoch {epoch+1}):")
            calculate_detailed_metrics(val_preds, val_targets, CONFIG['class_names'])
    
    # Training complete
    total_time = datetime.now() - total_start_time
    print(f"\n🎉 Progressive Training Complete!")
    print(f"Total time: {total_time}")
    print(f"Best validation accuracy: {best_val_acc:.2f}%")
    
    # Plot training history
    plot_training_history(all_train_losses, all_val_losses, all_train_accs, all_val_accs)
    
    return {
        'train_losses': all_train_losses,
        'val_losses': all_val_losses,
        'train_accs': all_train_accs,
        'val_accs': all_val_accs,
        'best_val_acc': best_val_acc,
        'total_time': total_time,
        'stage1_epochs': stage1_epochs
    }

print("Progressive training function defined!")

## Training Loop Implementation

The following training loop includes:
- **Comprehensive metrics tracking**: Accuracy, precision, recall, F1-score per class
- **GPU optimization**: Efficient batch processing with mixed precision support
- **Medical image specific validation**: Class-wise performance monitoring
- **Robust checkpointing**: Save best models and training state
- **Advanced visualization**: Real-time training progress and confusion matrices

In [ ]:
def train_epoch(model, train_loader, criterion, optimizer, device, epoch):
    """Train for one epoch with detailed metrics and debugging"""
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    all_predictions = []
    all_targets = []
    error_count = 0

    # Progress tracking
    batch_losses = []

    for batch_idx, (images, targets, image_ids) in enumerate(train_loader):
        # Debug: Check for problematic batches
        if torch.any(torch.isnan(images)) or torch.any(torch.isinf(images)):
            print(f"Warning: NaN/Inf detected in batch {batch_idx}")
            continue

        images, targets = images.to(device), targets.to(device)

        # Zero gradients
        optimizer.zero_grad()

        try:
            # Forward pass
            outputs = model(images)

            # Check for NaN outputs
            if torch.any(torch.isnan(outputs)):
                print(f"Warning: NaN outputs in batch {batch_idx}")
                continue

            loss = criterion(outputs, targets)

            # Check for NaN loss
            if torch.isnan(loss):
                print(f"Warning: NaN loss in batch {batch_idx}")
                continue

            # Backward pass
            loss.backward()

            # Gradient clipping to prevent exploding gradients
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

            optimizer.step()

            # Statistics
            running_loss += loss.item()
            batch_losses.append(loss.item())
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()

            # Store for detailed metrics
            all_predictions.extend(predicted.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())

            # Print progress every 20 batches with more details
            if batch_idx % 20 == 0 and batch_idx > 0:
                avg_loss = running_loss / (batch_idx + 1)
                accuracy = 100. * correct / total
                print(f'  Batch {batch_idx}/{len(train_loader)}: Loss: {avg_loss:.4f}, Acc: {accuracy:.2f}%, Errors: {error_count}')

                # Debug: Show unique predictions and targets in this batch
                unique_preds = np.unique(predicted.cpu().numpy())
                unique_targets = np.unique(targets.cpu().numpy())
                print(f'    Batch predictions: {unique_preds}, targets: {unique_targets}')

        except Exception as e:
            print(f"Error in batch {batch_idx}: {e}")
            error_count += 1
            continue

    # Calculate final metrics
    if len(batch_losses) > 0:
        epoch_loss = running_loss / len(batch_losses)
    else:
        epoch_loss = float('inf')

    epoch_acc = 100. * correct / total if total > 0 else 0.0

    print(f"  Epoch complete: {error_count} batch errors out of {len(train_loader)} batches")
    return epoch_loss, epoch_acc, all_predictions, all_targets

def validate_epoch(model, val_loader, criterion, device):
    """Validate for one epoch with detailed metrics"""
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    all_predictions = []
    all_targets = []
    all_probabilities = []

    with torch.no_grad():
        for images, targets, image_ids in val_loader:
            images, targets = images.to(device), targets.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, targets)

            # Get probabilities
            probabilities = F.softmax(outputs, dim=1)

            # Statistics
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()

            # Store for detailed metrics
            all_predictions.extend(predicted.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())
            all_probabilities.extend(probabilities.cpu().numpy())

    # Calculate metrics
    epoch_loss = running_loss / len(val_loader)
    epoch_acc = 100. * correct / total

    return epoch_loss, epoch_acc, all_predictions, all_targets, all_probabilities

print("Training and validation functions defined!")

In [ ]:
def calculate_detailed_metrics(predictions, targets, class_names):
    """Calculate and display detailed classification metrics"""
    from sklearn.metrics import classification_report, confusion_matrix

    # Classification report
    # Fix: Explicitly specify labels to handle cases where some classes are missing in the subset
    report = classification_report(
        targets,
        predictions,
        target_names=class_names,
        labels=np.arange(len(class_names)), # Specify all possible class indices
        output_dict=True,
        zero_division=0 # Handle cases with zero division (e.g., no samples for a class)
    )

    # Print formatted report
    print("\nDetailed Classification Metrics:")
    print("=" * 60)
    for i, class_name in enumerate(class_names):
        # Check if the class is in the report (it should be with labels specified)
        if str(i) in report: # classification_report uses string keys for labels
            metrics = report[str(i)]
            # Handle cases where a class might have 0 support
            precision = metrics.get('precision', 0.0)
            recall = metrics.get('recall', 0.0)
            f1_score = metrics.get('f1-score', 0.0)
            support = metrics.get('support', 0)

            print(f"{class_name:>8}: P={precision:.3f}, R={recall:.3f}, F1={f1_score:.3f}, Support={int(support)}")
        else:
             # This case should ideally not happen after specifying labels, but as a fallback
             print(f"{class_name:>8}: P=N/A, R=N/A, F1=N/A, Support=0")


    print("-" * 60)
    # Ensure global metrics are accessed correctly
    print(f"{'Accuracy':>8}: {report['accuracy']:.3f}")
    print(f"{'Macro Avg':>8}: P={report['macro avg']['precision']:.3f}, R={report['macro avg']['recall']:.3f}, F1={report['macro avg']['f1-score']:.3f}")
    print(f"{'Weighted Avg':>8}: P={report['weighted avg']['precision']:.3f}, R={report['weighted avg']['recall']:.3f}, F1={report['weighted avg']['f1-score']:.3f}")

    return report

def plot_confusion_matrix(predictions, targets, class_names, title="Confusion Matrix"):
    """Plot confusion matrix with medical focus"""
    # Ensure confusion matrix is generated for all classes
    cm = confusion_matrix(targets, predictions, labels=np.arange(len(class_names)))

    plt.figure(figsize=(12, 10))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names, yticklabels=class_names)
    plt.title(f'{title}\n(Medical Image Classification)', fontsize=14)
    plt.xlabel('Predicted Diagnosis', fontsize=12)
    plt.ylabel('True Diagnosis', fontsize=12)
    plt.xticks(rotation=45)
    plt.yticks(rotation=0)

    # Add accuracy for each class
    # Avoid division by zero if a class has no true samples
    class_sums = cm.sum(axis=1)
    class_accuracies = np.divide(cm.diagonal(), class_sums, out=np.zeros_like(cm.diagonal(), dtype=float), where=class_sums!=0)

    for i, acc in enumerate(class_accuracies):
        # Only display accuracy if the class exists in the true labels
        if class_sums[i] > 0:
             plt.text(len(class_names) + 0.1, i + 0.5, f'Acc: {acc:.2f}',
                    va='center', fontweight='bold')
        else:
             plt.text(len(class_names) + 0.1, i + 0.5, f'Acc: N/A',
                    va='center', fontweight='bold', color='gray')


    plt.tight_layout()
    plt.show()

    return cm

def plot_training_history(train_losses, val_losses, train_accs, val_accs):
    """Plot training history with medical context"""
    epochs = range(1, len(train_losses) + 1)

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

    # Loss plot
    ax1.plot(epochs, train_losses, 'b-', label='Training Loss', linewidth=2)
    ax1.plot(epochs, val_losses, 'r-', label='Validation Loss', linewidth=2)
    ax1.set_title('Model Loss (Medical Image Classification)', fontsize=14)
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    ax1.legend()
    ax1.grid(True, alpha=0.3)

    # Accuracy plot
    ax2.plot(epochs, train_accs, 'b-', label='Training Accuracy', linewidth=2)
    ax2.plot(epochs, val_accs, 'r-', label='Validation Accuracy', linewidth=2)
    ax2.set_title('Model Accuracy (Skin Lesion Diagnosis)', fontsize=14)
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Accuracy (%)')
    ax2.legend()
    ax2.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

print("Metrics and visualization functions defined!")

In [ ]:
# # Main training loop with comprehensive monitoring
# def train_model():
#     """Complete training loop with medical image focus"""
#     print("Starting VGG16 Transfer Learning Training for Skin Lesion Classification")
#     print("=" * 80)

#     # Training history
#     train_losses = []
#     val_losses = []
#     train_accs = []
#     val_accs = []

#     best_val_acc = 0.0
#     start_time = datetime.now()

#     for epoch in range(CONFIG['num_epochs']):
#         epoch_start = datetime.now()
#         print(f"\nEpoch {epoch+1}/{CONFIG['num_epochs']}")
#         print("-" * 50)

#         # Training phase
#         print("Training phase...")
#         train_loss, train_acc, train_preds, train_targets = train_epoch(
#             model, train_loader, criterion, optimizer, device, epoch+1
#         )

#         # Validation phase
#         print("Validation phase...")
#         val_loss, val_acc, val_preds, val_targets, val_probs = validate_epoch(
#             model, val_loader, criterion, device
#         )

#         # Update learning rate
#         scheduler.step(val_loss)
#         current_lr = optimizer.param_groups[0]['lr']

#         # Store history
#         train_losses.append(train_loss)
#         val_losses.append(val_loss)
#         train_accs.append(train_acc)
#         val_accs.append(val_acc)

#         # Calculate epoch time
#         epoch_time = datetime.now() - epoch_start

#         # Print epoch summary
#         print(f"\nEpoch {epoch+1} Summary:")
#         print(f"  Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
#         print(f"  Val Loss:   {val_loss:.4f} | Val Acc:   {val_acc:.2f}%")
#         print(f"  Learning Rate: {current_lr:.2e}")
#         print(f"  Time: {epoch_time.total_seconds():.1f}s")

#         # Check if best model
#         if val_acc > best_val_acc:
#             best_val_acc = val_acc
#             print(f"  New best validation accuracy: {best_val_acc:.2f}%")

#             # Save best model (we'll implement this next)
#             best_model_state = model.state_dict().copy()

#         # Early stopping check
#         if early_stopping(val_loss, model):
#             print(f"\nEarly stopping triggered after {epoch+1} epochs")
#             break

#         # Show detailed metrics every 5 epochs
#         if (epoch + 1) % 5 == 0:
#             print(f"\nDetailed Validation Metrics (Epoch {epoch+1}):")
#             calculate_detailed_metrics(val_preds, val_targets, CONFIG['class_names'])

#     # Training complete
#     total_time = datetime.now() - start_time
#     print(f"\nTraining Complete!")
#     print(f"Total time: {total_time}")
#     print(f"Best validation accuracy: {best_val_acc:.2f}%")

#     # Plot training history
#     plot_training_history(train_losses, val_losses, train_accs, val_accs)

#     return {
#         'train_losses': train_losses,
#         'val_losses': val_losses,
#         'train_accs': train_accs,
#         'val_accs': val_accs,
#         'best_val_acc': best_val_acc,
#         'total_time': total_time
#     }

# print("Main training function defined!")

In [ ]:
def validate_dataset(dataset, num_samples=10):
    """Validate that the dataset is working properly"""
    print("Validating dataset...")

    for i in range(min(num_samples, len(dataset))):
        try:
            image, label, image_id = dataset[i]
            print(f"Sample {i}: Image shape: {image.shape}, Label: {label}, ID: {image_id}")

            # Check for problematic images
            if torch.any(torch.isnan(image)):
                print(f"  WARNING: NaN values in image {image_id}")
            if torch.any(torch.isinf(image)):
                print(f"  WARNING: Inf values in image {image_id}")
            if torch.all(image == 0):
                print(f"  WARNING: All-zero image {image_id}")

        except Exception as e:
            print(f"  ERROR loading sample {i}: {e}")

print("Dataset validation complete!")

### Execute Progressive Training


In [ ]:
print("ISIC 2019 Skin Lesion Classification - Enhanced Training")
print("Model: EfficientNet-B3 with Progressive Training")
print(f"Dataset: {len(train_dataset)} train, {len(val_dataset)} val, {len(test_dataset)} test")
print(f"Device: {device}")
print(f"Image size: {CONFIG['image_size']}x{CONFIG['image_size']}")
print("\n" + "="*80)

# Validate datasets
validate_dataset(train_dataset, num_samples=5)

# Execute progressive training
training_history = train_model_progressive()

# Save the best model
print("\nSaving best model...")
best_checkpoint_path = save_model_checkpoint(
    model, None, None,  # Optimizer/scheduler not needed for final save
    epoch=len(training_history['train_losses']),
    val_acc=training_history['best_val_acc'],
    val_loss=min(training_history['val_losses']),
    config=CONFIG,
    filename="efficientnet_b3_progressive_best_model.pth"
)

print(f"\nProgressive Training Completed!")
print(f"Best validation accuracy: {training_history['best_val_acc']:.2f}%")
print(f"Total training time: {training_history['total_time']}")
print(f"Stage 1 epochs: {training_history['stage1_epochs']}")
print(f"Total epochs: {len(training_history['train_losses'])}")

## Model Evaluation and Testing

In [ ]:
def evaluate_model_comprehensive(model, test_loader, device, class_names):
    """Comprehensive model evaluation with medical focus"""
    print("Comprehensive Model Evaluation")
    print("=" * 50)

    model.eval()
    all_predictions = []
    all_targets = []
    all_probabilities = []
    all_image_ids = []

    with torch.no_grad():
        for images, targets, image_ids in test_loader:
            images, targets = images.to(device), targets.to(device)

            # Forward pass
            outputs = model(images)
            probabilities = F.softmax(outputs, dim=1)
            _, predicted = torch.max(outputs, 1)

            # Store results
            all_predictions.extend(predicted.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())
            all_probabilities.extend(probabilities.cpu().numpy())
            all_image_ids.extend(image_ids)

    # Calculate overall accuracy
    accuracy = accuracy_score(all_targets, all_predictions)
    print(f"Overall Test Accuracy: {accuracy*100:.2f}%")

    # Detailed metrics
    print("\nDetailed Classification Report:")
    report = calculate_detailed_metrics(all_predictions, all_targets, class_names)

    # Confusion matrix
    print("\nConfusion Matrix:")
    cm = plot_confusion_matrix(all_predictions, all_targets, class_names, "Test Set Confusion Matrix")

    # Per-class analysis
    print("\nPer-Class Performance Analysis:")
    for i, class_name in enumerate(class_names):
        class_mask = np.array(all_targets) == i
        if np.sum(class_mask) > 0:
            class_predictions = np.array(all_predictions)[class_mask]
            class_accuracy = accuracy_score(np.array(all_targets)[class_mask], class_predictions)
            print(f"  {class_name}: {class_accuracy*100:.1f}% accuracy ({np.sum(class_mask)} samples)")

    # Most confident predictions
    confidence_scores = np.max(all_probabilities, axis=1)
    correct_predictions = np.array(all_predictions) == np.array(all_targets)

    print(f"\nConfidence Analysis:")
    print(f"  Average confidence: {np.mean(confidence_scores):.3f}")
    print(f"  Confidence on correct predictions: {np.mean(confidence_scores[correct_predictions]):.3f}")
    print(f"  Confidence on incorrect predictions: {np.mean(confidence_scores[~correct_predictions]):.3f}")

    return {
        'predictions': all_predictions,
        'targets': all_targets,
        'probabilities': all_probabilities,
        'image_ids': all_image_ids,
        'accuracy': accuracy,
        'report': report,
        'confusion_matrix': cm
    }

def analyze_misclassifications(model, test_loader, device, class_names, num_samples=9):
    """Analyze and visualize misclassified samples"""
    print("Misclassification Analysis")
    print("=" * 40)

    model.eval()
    misclassified_samples = []

    with torch.no_grad():
        for images, targets, image_ids in test_loader:
            images, targets = images.to(device), targets.to(device)

            outputs = model(images)
            probabilities = F.softmax(outputs, dim=1)
            _, predicted = torch.max(outputs, 1)

            # Find misclassified samples
            incorrect_mask = predicted != targets
            if incorrect_mask.any():
                for i in range(len(images)):
                    if incorrect_mask[i]:
                        misclassified_samples.append({
                            'image': images[i].cpu(),
                            'true_label': targets[i].item(),
                            'pred_label': predicted[i].item(),
                            'probabilities': probabilities[i].cpu().numpy(),
                            'image_id': image_ids[i]
                        })

            if len(misclassified_samples) >= num_samples:
                break

    # Visualize misclassifications
    if misclassified_samples:
        fig, axes = plt.subplots(3, 3, figsize=(15, 15))
        axes = axes.flatten()

        for i, sample in enumerate(misclassified_samples[:num_samples]):
            # Denormalize image
            img = denormalize(sample['image'])
            img = torch.clamp(img, 0, 1)
            img = img.permute(1, 2, 0).numpy()

            # Plot
            axes[i].imshow(img)
            true_class = class_names[sample['true_label']]
            pred_class = class_names[sample['pred_label']]
            confidence = sample['probabilities'][sample['pred_label']]

            axes[i].set_title(f'True: {true_class}\nPred: {pred_class} ({confidence:.2f})\nID: {sample["image_id"]}')
            axes[i].axis('off')

        plt.suptitle('Misclassified Samples Analysis', fontsize=16)
        plt.tight_layout()
        plt.show()

        print(f"Analyzed {len(misclassified_samples[:num_samples])} misclassified samples")
    else:
        print("No misclassifications found in the sample!")

print("Evaluation functions defined!")

## Model Saving and Loading Utilities

In [ ]:
def save_model_checkpoint(model, optimizer, scheduler, epoch, val_acc, val_loss, config, filename=None):
    """Save complete model checkpoint with metadata"""
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"vgg16_skinlesion_epoch_{epoch}_acc_{val_acc:.2f}_{timestamp}.pth"

    # Create models directory
    models_dir = "../models"
    os.makedirs(models_dir, exist_ok=True)
    filepath = os.path.join(models_dir, filename)

    # Prepare checkpoint data
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict(),
        'val_accuracy': val_acc,
        'val_loss': val_loss,
        'config': config,
        'class_names': CONFIG['class_names'],
        'model_architecture': 'VGG16SkinLesionClassifier',
        'timestamp': datetime.now().isoformat(),
        'device': str(device)
    }

    # Save checkpoint
    torch.save(checkpoint, filepath)
    print(f"Model checkpoint saved: {filename}")
    print(f"   Epoch: {epoch}, Val Acc: {val_acc:.2f}%, Val Loss: {val_loss:.4f}")

    return filepath

def load_model_checkpoint(filepath, model, optimizer=None, scheduler=None):
    """Load model checkpoint with full state restoration"""
    print(f"Loading model checkpoint: {filepath}")

    # Load checkpoint
    checkpoint = torch.load(filepath, map_location=device)

    # Restore model state
    model.load_state_dict(checkpoint['model_state_dict'])

    # Restore optimizer and scheduler if provided
    if optimizer and 'optimizer_state_dict' in checkpoint:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

    if scheduler and 'scheduler_state_dict' in checkpoint:
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])

    # Print checkpoint info
    print(f"Checkpoint loaded successfully!")
    print(f"   Epoch: {checkpoint['epoch']}")
    print(f"   Validation Accuracy: {checkpoint['val_accuracy']:.2f}%")
    print(f"   Validation Loss: {checkpoint['val_loss']:.4f}")
    print(f"   Saved on: {checkpoint['timestamp']}")

    return checkpoint

def save_training_results(training_history, evaluation_results, filename=None):
    """Save complete training and evaluation results"""
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"training_results_{timestamp}.pkl"

    results_dir = "../results"
    os.makedirs(results_dir, exist_ok=True)
    filepath = os.path.join(results_dir, filename)

    # Combine all results
    complete_results = {
        'training_history': training_history,
        'evaluation_results': evaluation_results,
        'config': CONFIG,
        'timestamp': datetime.now().isoformat(),
        'model_type': 'VGG16SkinLesionClassifier'
    }

    # Save results
    with open(filepath, 'wb') as f:
        pickle.dump(complete_results, f)

    print(f"Training results saved: {filename}")
    return filepath

def export_model_for_inference(model, filepath=None):
    """Export optimized model for inference"""
    if filepath is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filepath = f"../models/vgg16_skinlesion_inference_{timestamp}.pth"

    # Set model to evaluation mode
    model.eval()

    # Create inference package
    inference_package = {
        'model_state_dict': model.state_dict(),
        'class_names': CONFIG['class_names'],
        'num_classes': CONFIG['num_classes'],
        'image_size': CONFIG['image_size'],
        'transforms': {
            'mean': [0.485, 0.456, 0.406],
            'std': [0.229, 0.224, 0.225]
        },
        'model_type': 'VGG16SkinLesionClassifier',
        'export_timestamp': datetime.now().isoformat()
    }

    torch.save(inference_package, filepath)
    print(f"Inference model exported: {filepath}")
    return filepath

print("Model saving and loading utilities defined!")

## Execute Training and Evaluation

Ready to train your VGG16 model! The cells below will:

1. **Start the training process** with comprehensive monitoring
2. **Evaluate the trained model** on the test set
3. **Save the best model** and training results
4. **Analyze performance** with detailed medical image metrics

**Note**: Training may take 30-60 minutes depending on your GPU. The notebook includes early stopping to prevent overfitting.

In [ ]:
# Start training process
print("ISIC 2019 Skin Lesion Classification Training")
print("Model: VGG16 Transfer Learning")
print(f"Dataset: {len(train_dataset)} train, {len(val_dataset)} val, {len(test_dataset)} test")
print(f"Device: {device}")
print("\n" + "="*60)

# Validate datasets before training
validate_dataset(train_dataset, num_samples=5)
validate_dataset(val_dataset, num_samples=5)
validate_dataset(test_dataset, num_samples=5)

# Evaluate model on test set before training
print("\nEvaluating model on test set before training...")
evaluation_results = evaluate_model_comprehensive(
    model, test_loader, device, CONFIG['class_names']
)

# Execute training
training_history = train_model()

# Save the best model after training
print("\nSaving best model...")
best_checkpoint_path = save_model_checkpoint(
    model, optimizer, scheduler,
    epoch=len(training_history['train_losses']),
    val_acc=training_history['best_val_acc'],
    val_loss=min(training_history['val_losses']),
    config=CONFIG,
    filename="vgg16_skinlesion_best_model.pth"
)

print(f"\nTraining completed!")
print(f"Best validation accuracy: {training_history['best_val_acc']:.2f}%")
print(f"Total training time: {training_history['total_time']}")

In [ ]:
# Comprehensive model evaluation on test set
print("\nStarting Comprehensive Model Evaluation")
print("="*60)

evaluation_results = evaluate_model_comprehensive(
    model, test_loader, device, CONFIG['class_names']
)

# Analyze misclassifications
print("\nAnalyzing Misclassifications...")
analyze_misclassifications(model, test_loader, device, CONFIG['class_names'])

# Save complete results
results_path = save_training_results(training_history, evaluation_results)

# Export model for inference
inference_model_path = export_model_for_inference(model)

print(f"\nComplete Evaluation Finished!")
print(f"Test Accuracy: {evaluation_results['accuracy']*100:.2f}%")
print(f"Results saved to: {results_path}")
print(f"Inference model: {inference_model_path}")

# Feature extraction example
print(f"\nFeature Extraction Capability:")
print("The model can extract 512-dimensional features from the second-to-last layer.")
print("Use model.extract_features(images) for feature extraction.")

## Training Complete - Summary and Next Steps

### What We Accomplished:

**Complete VGG16 Transfer Learning Pipeline**
- Implemented medical-image optimized data preprocessing
- Created balanced training with class weights for imbalanced data
- Integrated with our GPU-optimized Azure pipeline
- Added comprehensive evaluation metrics

**Advanced Model Architecture**
- VGG16 backbone with frozen feature layers
- Custom classifier head optimized for 9-class skin lesion diagnosis
- Feature extraction capability from second-to-last layer
- Dropout regularization and early stopping

**Production-Ready Training Infrastructure**
- Automated checkpoint saving and loading
- Real-time training visualization and monitoring
- Comprehensive evaluation with confusion matrices
- Export functionality for deployment

### Next Steps:

1. **Model Optimization**: Fine-tune hyperparameters based on results
2. **Feature Analysis**: Use extracted features for additional ML models
3. **Deployment**: Deploy model using Azure Container Apps
4. **Clinical Validation**: Test on additional medical image datasets

### Generated Files:
- `../models/vgg16_skinlesion_best_model.pth` - Best trained model
- `../models/vgg16_skinlesion_inference_*.pth` - Inference-ready model
- `../results/training_results_*.pkl` - Complete training history and metrics